In [ ]:
# ORCID Member clientID and secret
ID=""
SEC=""

In [ ]:
import requests

Get an Access token
```
URL=https://orcid.org/oauth/token
  HEADER: Accept: application/json
  METHOD: POST
  DATA: 
    client_id=[Your public API client ID]
    client_secret=[Your public API secret]
    grant_type=client_credentials
    scope=/read-public
```

In [ ]:
URL="https://orcid.org/oauth/token"
HEADERS={"Accept": "application/json"}
METHOD="POST"
DATA={"client_id": ID, "client_secret": SEC,  "grant_type": "client_credentials", "scope": "/read-public"}


In [ ]:
res=requests.post(url=URL, headers=HEADERS, data=DATA)

In [ ]:
res

In [ ]:
bearer = res.json()['access_token']

Info to user summary info in ORCID
```
Method: GET
Headers:
  Accept: application/json
  Authorization: Bearer [Stored access token]
URL: https://api.orcid.org/v3.0/[ORCID iD]/summary
```

In [ ]:
file_path = 'orcids.csv'
orcids = []

# Reading all lines from the file
try:
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines into a list
        for line in lines:
            orcids.append(line.strip())  # Print each line, stripping newline characters
except FileNotFoundError:
    print(f"The file at {file_path} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
len(orcids)

In [ ]:
headers = {"Accept": "application/json", "Authorization": "Bearer %s" % bearer}
url = "https://api.orcid.org/v3.0/%s/summary" % "0000-0002-5466-1432"
orc = requests.get(url=url, headers=headers)
orc.json()
# name_dict = orc.json()['name']
# name = name_dict['given-names']['value'] + ' ' + name_dict['family-name']['value'] 
# print(name)

In [ ]:
orcids[0]

In [ ]:
orgs = []
for orcid in orcids:
    headers = {"Accept": "application/json", "Authorization": "Bearer %s" % bearer}
    url = "https://api.orcid.org/v3.0/%s/summary" % orcid

    orc = requests.get(url=url, headers=headers)
    
    data=orc.json() 
    organization_names = []

    # Extract credit-name
    credit_name = data.get('credit-name', None)

    # Extract organization-name from employments
    employments = data.get('employments', {}).get('employment', [])
    if employments:
        organization_names = [employment.get('organization-name') for employment in employments if 'organization-name' in employment]    

    print(f"{orcid}, {credit_name}, {organization_names}")
    orgs.append([orcid, credit_name, organization_names])


In [ ]:
len(orgs) # should match length of orcid list

In [ ]:
orgs

In [ ]:
import csv
# Write to CSV
output_file = 'orcid_data.csv'

with open(output_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(['ORCID', 'Name', 'Organizations'])
    # Write data
    for row in orgs: 
        writer.writerow(row)

